# Exercise 4 - PyTorch Geometric Tutorial

This exercise is to introduce you to **[PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/)** (PyG).

PyG is a library built upon PyTorch to easily write and train Graph Neural Networks (GNNs).

*Note*: you may not use PyTorch Geometric for Homework 2, except for data loading!

## 0. To install the required packages, run the cell below.

**Note**: if you are running this notebook on your local machine and are using `conda`, you can comment out the `!pip install` lines below and instead run in your terminal:
```bash
conda install -y pyg -c pyg
```

In [ ]:
# Install required packages.
import os
import matplotlib.pyplot as plt
import torch
from torch import nn

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Node Classification with Graph Neural Networks

Last week, you implemented GNNs to perform edge prediction by implementing a prediction head that performed a dot-product between the node embeddings. This week, we will explore another problem: node classification.

<img src="img/node_class.png" width="600" />

To demonstrate the abilities of the library, we make use of the [`torch_geometric.datasets.PPI`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.PPI.html#torch_geometric.datasets.PPI) dataset, which contains **protein-protein interaction** networks from different human tissues. We have 20 networks for training, 2 networks for validation and 2 for testing. A node in each graph represents a protein and is described by a 50-dimensional feature vector reflecting different immunological signatures. The task is to predict protein roles in 121 different gene ontology sets. Thus, for each node $v$ we have label vector $y_v \in \{0, 1\}^{121}$ leading to 121 binary classification problems.

This is a different problem than you saw last week! Indeed, instead of having a single graph where we want to predict part of it, we here have multiple and want to do prediction on new sets of graphs.   
$\rightarrow$ We consider **inductive learning**, *i.e*, we are given the training set of graphs with ground-truth labels for each node in the graphs, and we want to infer the labels for the nodes of the test graphs.

## 1. Dataset and DataLoader

First of all, load the train and test split of the PPI dataset and inspect them.
[`torch_geometric.datasets.PPI`](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/ppi.html) and [`torch_geometric.loader.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) might be useful.

To enable fast prototyping and testing let's use only the first 3 graphs for training. At home, you can use all 20 training graphs to get better performance.
To easily create a subset of a dataset, you may want to use [`torch.utils.data.Subset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset).

In [ ]:
from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader

dataset_train = PPI(...)  # TODO
dataset_train = torch.utils.data.Subset(...)  # TODO
trainloader = ...  # TODO
dataset_test = PPI(...)  # TODO
testloader = ...  # TODO

print()
print(f'Train set: {dataset_train.dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset_train)}')
print(f'Number of features: {dataset_train.dataset.num_features}')
print(f'Number of classes: {dataset_train.dataset.num_classes}')

print()
print(f'First element of the train set:\n{dataset_train[0]}')
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {dataset_train[0].num_nodes}')
print(f'Number of edges: {dataset_train[0].num_edges}')
print(f'Average node degree: {dataset_train[0].num_edges / dataset_train[0].num_nodes:.2f}')
print(f'Has isolated nodes: {dataset_train[0].has_isolated_nodes()}')
print(f'Has self-loops: {dataset_train[0].has_self_loops()}')
print(f'Is undirected: {dataset_train[0].is_undirected()}')


print()
print(f'First element of the test set:\n{dataset_test[0]}')
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {dataset_test[0].num_nodes}')
print(f'Number of edges: {dataset_test[0].num_edges}')
print(f'Average node degree: {dataset_test[0].num_edges / dataset_test[0].num_nodes:.2f}')
print(f'Has isolated nodes: {dataset_test[0].has_isolated_nodes()}')
print(f'Has self-loops: {dataset_test[0].has_self_loops()}')
print(f'Is undirected: {dataset_test[0].is_undirected()}')

You can access the node embedding using `.x`, the edge indices using `.edge_index`, and the target class labels using `.y`. See below:

In [ ]:
data = dataset_train[0]
print(f'The node features have shape {data.x.shape}')
print(f'The edge indices have shape {data.edge_index.shape}')
print(f'The target labels have shape {data.y.shape}')

## 2. Simple Random prediction baseline

As a very simple baseline, let's consider random predictions, *i.e.*, predicting 0 or 1 with probability 0.5.

In [ ]:
class SimpleRandomBaseline(torch.nn.Module):
    """Simple baseline that randomly assigns a label to each node."""

    def __init__(self, num_tasks):
        """
        Initialize the baseline.
        
        Args:
            num_tasks (int): Number of tasks (classes).
        """
        super().__init__()
        ...  # TODO

    def forward(self, x):
        """
        Random prediction for each node.

        Args:
            x (Tensor): Input node features of shape (num_nodes, in_features).

        Returns:
            Tensor: Predicted labels of shape (num_nodes, num_tasks).
        """
        return ...  # TODO

In [ ]:
m1 = SimpleRandomBaseline(dataset_train.dataset.num_classes)

## 3. Metric

For each node, we have 121 binary classification problems. To assess the quality of the model, we will use micro-averaged F1 score, *i.e.*, the overall F1 score considering the total true/false positives/negatives. As a reminder, the F1 score for a single class is computed as follow:
$$
F_1 = \frac{2 \, \mathrm{Precision} \cdot \mathrm{Recall}}{\mathrm{Precision} + \mathrm{Recall}} = \frac{2TP}{2TP + FP + FN},
$$
with $TP$ the true positives, $FP$ false positives, and $FN$ false negatives. See more details on [Wikipedia](https://en.wikipedia.org/wiki/F-score#Dependence_of_the_F-score_on_class_imbalance).

Below, compute the micro-averaged F1 score of the random baseline on the two graphs of the test set. You may use existing implementation, *e.g.*, [`sklearn.metrics.f1_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html).

*Hint*: for the above random baseline, the F1 score should be around $\sim0.37$.

In [ ]:
# TODO: compute the micro-averaged F1 score on the random baseline.

## 4. Simple linear model on raw node features

As a slightly less naive baseline, let's consider a linear classifier operating directly on the raw node features.

Implement such a model and compute its performance. You may use existing implementations, *e.g.*, [`sklearn.linear_model.LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and [`sklearn.multioutput.MultiOutputClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) might be useful.

*Hint*: for a simple linear model operating directly on the raw node features, the F1 score should be around $\sim0.44$.

In [ ]:
# TODO: prepare and train a linear classifier on the node features.

In [ ]:
# TODO: compute the micro-averaged F1 score on the linear classifier.

## 5. Multi-layer Perception Network on node features

In theory, we should be able to infer the protein function solely based on its feature representation, without taking any information about interaction between proteins into account.

Let's verify that by constructing a simple MLP that solely operates on input node features. Implement a simple MLP below, with two layers and ReLU activations.

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F


class MLP(torch.nn.Module):
    """Simple MLP with two hidden layers."""

    def __init__(self, in_channels, hidden_channels, out_channels):
        """
        Initialize the MLP.
        
        Args:
            in_channels (int): Number of input node features.
            hidden_channels (int): Number of hidden units.
            out_channels (int): Number of output features.
        """
        super().__init__()
        ...  # TODO

    def forward(self, x, edge_index=None):
        """
        Compute the node predictions.

        Args:
            x (Tensor): Input node features of shape (num_nodes, in_channels).
            edge_index (LongTensor): Graph edge indices of shape (2, num_edges).
                It is here for compatibility issue, but is not used.
        
        Returns:
            Tensor: Output node features of shape (num_nodes, out_channels).
        """
        # edge_index is for compatibility, it's actually not used here.
        ...  # TODO
        return ...  # TODO

Let's implement a `train` **function** to train a model and a `test` **function** to evaluate it.

Do no hesitate to come back to the previous exercise if you need, a lot of this sort of implementation is similar throughout projects in PyTorch!

In [ ]:
def train(model, optimizer, criterion, dataloader):
    """
    Train the model for one epoch on the dataloader.
    
    You should return the average loss over the dataloader (look into running loss).
    """
    ...  # TODO
    return ...  # TODO

def test(model, dataloader):
    """Test the model on the dataloader and return the F1 score."""
    ...  # TODO
    return ...  # TODO

Our MLP is defined by two linear layers and enhanced by [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html?highlight=relu#torch.nn.ReLU) non-linearity.
Here, we first map the 50-dimensional feature vector to a hidden embedding (`hidden_channels=256`), while the second linear layer acts as a classifier that should map each low-dimensional node embedding to 121 binary classifiers.

Test this simple MLP. As the loss function, we use the **binary cross entropy** (with logits) and as the optimizer, we use **Adam**.

*Hint*: the layers above should give marginally better results than a simple linear model.

In [ ]:
model = MLP(...)  # TODO

criterion = torch.nn.BCEWithLogitsLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(...)  # TODO: Define optimizer.

loss = []
for epoch in range(1, 1001):
    loss.append(
        train(model, optimizer, criterion, trainloader)
    )
    print(f'Epoch: {epoch:03d}, Loss: {loss[-1]:.4f}')

fig = plt.figure()
plt.title('Training loss')
plt.plot(loss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
# Evaluate the performance of the model.
train_acc = test(model, trainloader)
print(f'Train F1: {train_acc:.4f}')

test_acc = test(model, testloader)
print(f'Test F1: {test_acc:.4f}')

## 6. Graph Isomorphism Network on node features with protein-protein interaction information

Now, let's use GNNs to make use of the graph structure! Let's start with [`torch.geometric.nn.GIN`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.GIN.html#torch_geometric.nn.models.GIN), but do not hesitate to explore [`torch_geometric.nn`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#models) to test various models.

In [ ]:
from torch_geometric.nn import GIN

model = GIN(
    in_channels=dataset_train.dataset.num_features, 
    hidden_channels=256, 
    num_layers=2, 
    out_channels=dataset_train.dataset.num_classes
)
print(model)
criterion = torch.nn.BCEWithLogitsLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(...)  # TODO: Define optimizer.

loss = []
for epoch in range(1, 1001):
    loss.append(
        train(model, optimizer, criterion, trainloader)
    )
    print(f'Epoch: {epoch:03d}, Loss: {loss[-1]:.4f}')

fig = plt.figure()
plt.title('Training loss')
plt.plot(loss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
# Evaluate the performance of the model.
train_acc = test(model, trainloader)
print(f'Train F1: {train_acc:.4f}')

test_acc = test(model, testloader)
print(f'Test F1: {test_acc:.4f}')

Compare the performances of the baseline to the MLP and GNN. How do they compare?

*Hint*: as the GNN is more powerful, it may overfit more. However, you should still observe an improvement as it considers the connectivity between the nodes, *e.g.*, a test F1 score $>0.55$.

We trained with only 3 graphs out of the 20 of the training set. At home and when you have time, try to increase the number of graphs you train on, and you should see an improvement of the performance on the test set.

## Conclusion

In this exercise, you have seen how to apply GNNs to real-world problems using *PyTorch Geometric* library, and, in particular, how they can effectively be used for boosting a model's performance when graph structure is taken into account.

## (Optional) Exercises

1. To achieve better model performance and to avoid overfitting, it is usually a good idea to select the best model based on an additional validation set.
The `PPI` dataset provides a validation graph set as `split="val"`. Try hyperparameter search!

2. *PyTorch Geometric* has many different GNN architectures. Try to play with them!

3. We trained only on 3 training graphs, but the whole dataset contains 20 graphs for the training. Try to train on the whole dataset to boost the performance!

4. Visualize learnt node embeddings with respect one of the binary tasks to see whether embeddings have well clustered structure!

## (Optional) GIN implementation
Let's try to implement the GIN network ourselves in PyTorch.

To do so, we need to implement:
1. the [`GINConv`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GINConv.html#torch_geometric.nn.conv.GINConv) convolutional layer,
2. a prediction head operating at the node level, which we will implement with a simple linear transformation of the node embeddings,
3. and the entire model which will consist in some `GINConv` followed by a prediction head.

For the `GINConv`, the formula in matrix form is given by:
$$
H^{(l+1)} = h_{\Theta} \left( \left(\mathbf{A} + (1+\epsilon)\,\mathbf{I} \right) \cdot H^{(l)}\right),
$$
where $h_{\Theta}$ is an MLP with two layers and ReLU activation function, $\mathbf{A}$ is the adjacency matrix, $\mathbf{I}$ the identity matrix, and $H^{(l)}$ the node embeddings of layer $l$.

Implement it below for `MyGINConv`:

In [ ]:
class MyGINConv(nn.Module):
    """GIN convolutional layer."""

    def __init__(self, in_features, out_features, epsilon=0.0):
        """
        Initialize the GIN convolutional layer.
        
        Args:
            in_features (int): number of input node features.
            out_features (int): number of output node features.
            epsilon (float): epsilon value. (default=0.0)
        """
        super().__init__()
        ...  # TODO

    def forward(self, x, adj):
        """
        Perform graph convolution operation.

        Args:
            x (Tensor): Input node features of shape (num_nodes, in_features).
            adj (Tensor): Adjacency matrix of the graph, shape (num_nodes, num_nodes).

        Returns:
            Tensor: Output node features after graph convolution, shape (num_nodes, out_features).
        """
        ...  # TODO
        return ...  # TODO

For the prediction head, we can directly make the predictions $\hat{\mathbf{y}}_v$ from the node embeddings $\mathbf{h}_v^{(l)}$. Let's implement it by doing a simple linear transformation:
$$
\hat{\mathbf{y}}_v = \mathbf{W}\mathbf{h}_v^{(l)} + \mathbf{b},
$$

In [ ]:
class LinearHead(nn.Module):
    """Prediction head using a linear transformation of the node embedding."""

    def __init__(self, in_features, num_tasks):
        """
        Initialize the prediction head.

        Args:
            in_features (int): number of input node features.
            num_tasks (int): number of prediction tasks (classes).
        """
        super().__init__()
        ...  # TODO

    def forward(self, x):
        """
        Node-level prediction given the node embeddings.

        Args:
            x (Tensor): Input node features of shape (num_nodes, in_features).

        Returns:
            Tensor: Node predictions of shape (num_nodes, num_tasks).
        """
        return ...  # TODO

As last week, let's aggregate them into a single model that we call `MyGIN`.

To be compatible with the code above, the model's forward method will take as input edge indices `edge_index`, however our `GINConv` above uses adjacency matrices.
Therefore, we will simply convert the edge indices into an adjacency matrix in the forward pass.

In [ ]:
def edge_index_to_adj(edge_index, num_nodes):
    """
    Convert edge index to adjacency matrix.

    Args:
        edge_index (LongTensor): Graph edge indices of shape (2, num_edges).
        num_nodes (int): Number of nodes in the graph.

    Returns:
        Tensor: Adjacency matrix of shape (num_nodes, num_nodes).
    """
    adj = torch.zeros((num_nodes, num_nodes), device=edge_index.device)
    adj[edge_index[0], edge_index[1]] = 1.0
    return adj


class MyGIN(nn.Module):
    """Re-implement the GIN network for node classification."""

    def __init__(self, in_channels, hidden_channels, num_layers, out_channels, epsilon=0.0):
        """
        Initialize the GNN model for node classification.

        Args:
            in_channels (int): Number of input node features.
            hidden_channels (int): Number of hidden units.
            num_layers (int): Number of GINConv layers.
            out_channels (int): Number of prediction tasks (classes).
            epsilon (float): Epsilon value for GINConv. (default=0.0)
        """
        super().__init__()
        ...  # TODO

    def forward(self, x, edge_index):
        """
        Perform forward pass for node classification.

        Args:
            x (Tensor): Input node features of shape (num_nodes, num_features).
            adj (Tensor): Adjacency matrix of the graph, shape (num_nodes, num_nodes).

        Returns:
            Tensor: Predicted edge probabilities for each pair of nodes, shape (num_nodes, num_nodes).
        """
        ...  # TODO

        return ...  # TODO

Try to use it above and see if it works! However, note that it will be slower than the PyTorch Geometric implementation, so you may want to reduce the epochs.